In [1]:
!pip install -q -U transformers==4.47.0
!pip install --upgrade "transformers>=4.47.0" "peft>=0.9.0"
!pip install -q  accelerate==0.26.0
!pip install --upgrade "accelerate>=0.27.0"
!pip install --prefer-binary bitsandbytes
!pip install -U bitsandbytes
!python -m bitsandbytes
!nvidia-smi


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 124.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 149.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 128.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 13.9 MB/s eta 0:00:00
  Attempting u

In [2]:
from google.colab import drive
import sys,os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
drive.mount('/content/drive')

modules_dir = '/content/drive/MyDrive/lllava_num_pec_imprv_github/from_google_drive/Modules'
sys.path.append(modules_dir)
from llava_utils import *
from data_utils import *
from vit_finetuning_utils import *



Mounted at /content/drive


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/674 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

In [3]:
# Getting the vision_transformer

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)
model_name = "llava-hf/llava-1.5-7b-hf"
processor,base_model = get_inference_ready_model(model_name, None , None , quantization_config, load_from_local=False, local_model_path=None, no_quantize=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = processor.tokenizer

baseline_vit = get_vit_and_print_config(base_model)
baseline_vit  = prepare_model_for_kbit_training(baseline_vit )




config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

CLIPVisionTransformer(
  (embeddings): CLIPVisionEmbeddings(
    (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (position_embedding): Embedding(577, 1024)
  )
  (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (encoder): CLIPEncoder(
    (layers): ModuleList(
      (0-23): 24 x CLIPEncoderLayer(
        (self_attn): CLIPAttention(
          (k_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear4bit(in_features=1024, out_features=1024, bias=True)
        )
        (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): CLIPMLP(
          (activation_fn): QuickGELUActivation()
          (fc1): Linear4bit(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear4bit(in_features=4096,

In [4]:
# Replicating the vit transform from the thesis
vit_transform = transforms.Compose([
    transforms.ColorJitter(
        brightness=0.2,
        contrast=0.2,
        saturation=0.2,
        hue=0.05
    ),
    transforms.RandomAffine(
        degrees=10,        # ±10° rotation
        scale=(0.95, 1.05) # scale range
    ),
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.GaussianBlur(
        kernel_size=3,
        sigma=(0.1, 1.0)
    ),
])

In [11]:
def load_config_vit(config_path: str):
    with open(config_path, "r") as f:
        cfg = json.load(f)

    vit_lora_config = ViTLoRAConfig(
        r=cfg["peft"]["r"],
        lora_alpha=cfg["peft"]["lora_alpha"],
        target_modules=cfg["peft"]["target_modules"],
    )

    vit_training_config = ViTTrainingConfig(
        learning_rate=cfg["training"]["learning_rate"],
        weight_decay=cfg["training"]["weight_decay"],
        total_epochs=cfg["training"]["total_epochs"],
        batch_size_train=cfg["training"]["batch_size_train"],
        batch_size_val=cfg["training"]["batch_size_val"],
        mixed_precision=cfg["training"]["mixed_precision"],
        warmup_steps=cfg["training"]["warmup_steps"],
        patience=cfg["training"]["patience"],
        freeze_vit=cfg["classifier"]["freeze_vit"],
    )

    vit_dataset_config = ViTDatasetConfig(
        dataset_name=cfg["dataset"]["train_name"],
        categories_train=cfg["dataset"]["categories"]["train"],
        categories_val=cfg["dataset"]["categories"]["val"],
        categories_test=cfg["dataset"]["categories"]["test"],
        nums=cfg["dataset"]["nums"],
        aug_factor_train=cfg["dataset"]["aug_factor_train"],
        undersample_ratio=cfg["dataset"]["undersample_ratio"],
    )

    vit_experiment_config = ViTExperimentConfig(
        save_path=cfg["experiment"]["save_path"],
        num_classes=cfg["classifier"]["num_classes"],
    )

    print("=" * 60)
    print("VIT CONFIGURATION SUMMARY")
    print("=" * 60)

    print("\n📊 DATASET")
    print(f"  Dataset: {vit_dataset_config.dataset_name}")
    print(f"  Train split: {vit_dataset_config.train_split}")
    print(f"  Train categories: {vit_dataset_config.categories_train}")
    print(f"  Val categories: {vit_dataset_config.categories_val}")
    print(f"  Test categories: {vit_dataset_config.categories_test}")
    print(f"  Numbers: {vit_dataset_config.nums}")
    print(f"  Aug factor (train): {vit_dataset_config.aug_factor_train}")

    print("\n🎯 TRAINING")
    print(f"  LR: {vit_training_config.learning_rate}")
    print(f"  Weight decay: {vit_training_config.weight_decay}")
    print(f"  Epochs: {vit_training_config.total_epochs}")
    print(f"  Batch size (train): {vit_training_config.batch_size_train}")
    print(f"  Batch size (val): {vit_training_config.batch_size_val}")
    print(f"  Mixed precision: {vit_training_config.mixed_precision}")
    print(f"  Warmup steps: {vit_training_config.warmup_steps}")
    print(f"  Patience: {vit_training_config.patience}")
    print(f"  Freeze ViT: {vit_training_config.freeze_vit}")

    print("\n🔧 PEFT (LoRA)")
    print(f"  r: {vit_lora_config.r}")
    print(f"  alpha: {vit_lora_config.lora_alpha}")
    print(f"  target modules: {', '.join(vit_lora_config.target_modules)}")

    print("\n💾 EXPERIMENT")
    print(f"  Save path: {vit_experiment_config.save_path}")
    print(f"  Num classes: {vit_experiment_config.num_classes}")

    print("=" * 60)

    return (
        vit_dataset_config,
        vit_training_config,
        vit_lora_config,
        vit_experiment_config,
    )

In [13]:
def load_config_vit(config_path: str):
    import json

    with open(config_path, "r") as f:
        cfg = json.load(f)

    vit_lora_config = ViTLoRAConfig(
        r=cfg["peft"]["r"],
        lora_alpha=cfg["peft"]["lora_alpha"],
        target_modules=cfg["peft"]["target_modules"],
    )

    vit_training_config = ViTTrainingConfig(
        learning_rate=cfg["training"]["learning_rate"],
        weight_decay=cfg["training"]["weight_decay"],
        total_epochs=cfg["training"]["total_epochs"],
        batch_size_train=cfg["training"]["batch_size_train"],
        batch_size_val=cfg["training"]["batch_size_val"],
        mixed_precision=cfg["training"]["mixed_precision"],
        warmup_steps=cfg["training"]["warmup_steps"],
        patience=cfg["training"]["patience"],
        freeze_vit=cfg["classifier"]["freeze_vit"],
    )

    vit_dataset_config = ViTDatasetConfig(
        dataset_name=cfg["dataset"]["train_name"],
        train_split=cfg["dataset"]["train_name"],
        categories_train=cfg["dataset"]["train_objs"],
        categories_val=cfg["dataset"]["val_objs"],
        categories_test=cfg["dataset"]["test_objs"],
        nums=[1,2,3,4,5,6,7,8,9,10],  # JSON does not have "nums"
        aug_factor_train=cfg["dataset"]["augment_factor"],
        undersample_ratio=1.0,  # JSON does not have "undersample_ratio"
    )

    vit_experiment_config = ViTExperimentConfig(
        save_path=cfg["paths"]["results_save"],
        num_classes=cfg["classifier"]["num_classes"],
    )

    print("=" * 60)
    print("VIT CONFIGURATION SUMMARY")
    print("=" * 60)

    print("\n📊 DATASET")
    print(f"  Dataset: {vit_dataset_config.dataset_name}")
    print(f"  Train split: {vit_dataset_config.train_split}")
    print(f"  Train categories: {vit_dataset_config.categories_train}")
    print(f"  Val categories: {vit_dataset_config.categories_val}")
    print(f"  Test categories: {vit_dataset_config.categories_test}")
    print(f"  Numbers: {vit_dataset_config.nums}")
    print(f"  Aug factor (train): {vit_dataset_config.aug_factor_train}")

    print("\n🎯 TRAINING")
    print(f"  LR: {vit_training_config.learning_rate}")
    print(f"  Weight decay: {vit_training_config.weight_decay}")
    print(f"  Epochs: {vit_training_config.total_epochs}")
    print(f"  Batch size (train): {vit_training_config.batch_size_train}")
    print(f"  Batch size (val): {vit_training_config.batch_size_val}")
    print(f"  Mixed precision: {vit_training_config.mixed_precision}")
    print(f"  Warmup steps: {vit_training_config.warmup_steps}")
    print(f"  Patience: {vit_training_config.patience}")
    print(f"  Freeze ViT: {vit_training_config.freeze_vit}")

    print("\n🔧 PEFT (LoRA)")
    print(f"  r: {vit_lora_config.r}")
    print(f"  alpha: {vit_lora_config.lora_alpha}")
    print(f"  target modules: {', '.join(vit_lora_config.target_modules)}")

    print("\n💾 EXPERIMENT")
    print(f"  Save path: {vit_experiment_config.save_path}")
    print(f"  Num classes: {vit_experiment_config.num_classes}")

    print("=" * 60)

    return (
        vit_dataset_config,
        vit_training_config,
        vit_lora_config,
        vit_experiment_config,
    )

In [15]:
@dataclass
class ViTTrainingConfig:
    learning_rate: float
    weight_decay: float
    total_epochs: int
    batch_size_train: int
    batch_size_val: int
    mixed_precision: bool
    warmup_steps: int
    patience: int
    freeze_vit: bool

@dataclass
class ViTDatasetConfig:
    dataset_name: str
    train_split: str                   # JSON has train_name as str
    categories_train: Optional[List[str]]  # can be None
    categories_val: Optional[List[str]]    # can be None
    categories_test: Optional[List[str]]   # can be None
    nums: Optional[List[int]] = None       # not in JSON
    aug_factor_train: int = 1
    undersample_ratio: Optional[float] = None

@dataclass
class ViTLoRAConfig:
    r: int
    lora_alpha: float
    target_modules: List[str]

@dataclass
class ViTExperimentConfig:
    save_path: str
    num_classes: int

In [16]:
vit_config_path = '/content/drive/MyDrive/lllava_num_pec_imprv_github/downloaded_from_vm_instance/vit_train_config.json'
vit_dataset_config, vit_training_config, vit_lora_config, vit_experiment_config = load_config_vit(vit_config_path)


VIT CONFIGURATION SUMMARY

📊 DATASET
  Dataset: CLEVR
  Train split: CLEVR
  Train categories: None
  Val categories: None
  Test categories: None
  Numbers: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
  Aug factor (train): 3

🎯 TRAINING
  LR: 5e-05
  Weight decay: 0.0001
  Epochs: 16
  Batch size (train): 8
  Batch size (val): 32
  Mixed precision: True
  Warmup steps: 400
  Patience: 3
  Freeze ViT: False

🔧 PEFT (LoRA)
  r: 10
  alpha: 10
  target modules: MLP, att_v, att_q

💾 EXPERIMENT
  Save path: /content/drive/MyDrive/Modality Gap and Numerosity Thesis/Experiments/Exp6/vit_zoo/clevr_finetuned_2
  Num classes: 10


In [24]:
def prepare_vit_datasets_and_dataloaders(
    dataset_config,
    training_config,
    processor,
    vit_transform=None,
):
    DATASET_PATHS = {
        "CCNL": '/content/drive/MyDrive/Modality Gap and Numerosity Thesis/models/images',
        "CCNL2": '/content/drive/MyDrive/Modality Gap and Numerosity Thesis/CCNL2',
        "CLEVR": '/content/drive/MyDrive/Modality Gap and Numerosity Thesis/CLEVR',
        "CHLDBOOK": '/content/drive/MyDrive/Modality Gap and Numerosity Thesis/childbook',
    }

    if dataset_config.dataset_name not in DATASET_PATHS:
        raise ValueError(f"Unknown dataset: {dataset_config.dataset_name}")

    dataset_path = DATASET_PATHS[dataset_config.dataset_name]

    full_train_dataset = ImageDatasetVit(
        image_base_path=dataset_path,
        dataset=dataset_config.dataset_name,
        processor=processor,
        nums=dataset_config.nums,
        categories=dataset_config.categories_train,
        transform=vit_transform,
        aug_factor=dataset_config.aug_factor_train,
        undersamp_ratio=dataset_config.undersample_ratio,
    )

    dataset_size = len(full_train_dataset)
    train_size = int(0.8 * dataset_size)
    val_size = dataset_size - train_size

    indices = torch.randperm(dataset_size)
    train_indices = indices[:train_size]
    val_indices = indices[train_size:]

    train_dataset = Subset(full_train_dataset, train_indices)
    val_dataset = Subset(full_train_dataset, val_indices)

    test_dataset = ImageDatasetVit(
        image_base_path=dataset_path,
        dataset=dataset_config.dataset_name,
        processor=processor,
        nums=dataset_config.nums,
        categories=dataset_config.categories_test,
        transform=None,
        undersamp_ratio=1.0,
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=training_config.batch_size_train,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=training_config.batch_size_val,
        shuffle=False,
        num_workers=4,
        pin_memory=True,
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=training_config.batch_size_val,
        shuffle=False,
        num_workers=4,
        pin_memory=True,
    )

    return train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset

In [25]:
target_modules = target_module_list_creator(
    vit_lora_config.target_modules,
    target_levels=None
)

pefted_vit_finetuned_shallow = setup_peft_vit_mlp(
    baseline_vit,
    r=vit_lora_config.r,
    lora_alpha=vit_lora_config.lora_alpha,
    target_modules=target_modules
)

pefted_vit_finetuned_shallow_classifier = ViTClassifier(
    PeftViTWrapper(pefted_vit_finetuned_shallow),
    num_classes=vit_experiment_config.num_classes,
    freeze_vit=vit_training_config.freeze_vit
).to(device)

h_params_shallow = {
    'num_epochs': vit_training_config.total_epochs,
    'lr': vit_training_config.learning_rate,
    'weight_decay': vit_training_config.weight_decay,
    'mixed_prec_train': vit_training_config.mixed_precision,
    'on_cats': False,
    'warmup_steps': vit_training_config.warmup_steps,
    'patience': vit_training_config.patience
}
train_loader, val_loader, test_loader, train_dataset, val_dataset, test_dataset = prepare_vit_datasets_and_dataloaders(
    vit_dataset_config,
    vit_training_config,
    processor,vit_transform=None)

trainable params: 3,440,640 || all params: 306,948,096 || trainable%: 1.1209
🔧 ViTClassifier init: mode=cls, svd=False, feature_dim=1024, classifier_input=1024


In [27]:
train_losses, train_accs, val_losses, val_acc = train_vit_classifier(pefted_vit_finetuned_shallow_classifier,train_loader,val_loader,device,**h_params_shallow)
save_peft_vit_classifier(pefted_vit_finetuned_shallow_classifier,save_path)

KeyboardInterrupt: 